In [1]:
"""
This notebook illustrates the process of training a stacked convolutional autoencoder using the unlabeled data in the stl_10 
datasets, and then by attacing a fully connected layer and a final classifier, trained with the avaliable training set in 
the stl_10 dataset, to the trained encoder layers, thus building a classifier without huge number of training instances.
"""



import tensorflow as tf
import numpy as np
import math

In [ ]:
# function to build the computational graph for the convolutional autoencoder
def autoencoder(input_shape=[None,96,96,3],
               n_filters=[3,96,256,256],
               filter_sizes=[2,2,2],
            ):
    x_tensor=tf.placeholder(tf.float32,input_shape,name='x')
    
    
    current_input=x_tensor
        # Build the encoder
    encoder=[]
    shapes=[]
    
    for layer_i, n_output in enumerate(n_filters[1:]):
        with tf.name_scope('encode_{}'.format(layer_i)):
            n_input=current_input.get_shape().as_list()[3]
            shapes.append(current_input.get_shape().as_list())
            w=tf.Variable(
            tf.random_uniform([filter_sizes[layer_i],filter_sizes[layer_i],
                          n_input,n_output],
                         -1.0/math.sqrt(n_input),
                         1.0/math.sqrt(n_input)),name='weights')
            b=tf.Variable(tf.zeros([n_output]),name='biases')
            encoder.append(w)
            output=tf.nn.elu(
            tf.add(tf.nn.conv2d(current_input,w,strides=[1,2,2,1],padding='SAME'),b))
        
            current_input = output
        
            z=current_input
    encoder.reverse()
    shapes.reverse()
    for layer_i,shape in enumerate(shapes):
        with tf.name_scope('decode_{}'.format(layer_i)):
            w=encoder[layer_i]
            b=tf.Variable(tf.zeros([w.get_shape().as_list()[2]]))
            output=tf.nn.elu(tf.add(tf.nn.conv2d_transpose(current_input,w,
                                                           tf.stack([tf.shape(x_tensor)[0],
                                                                     shape[1],
                                                                     shape[2],
                                                                     shape[3]]),
                                                           strides=[1,2,2,1],
                                                           padding='SAME'),b))
            current_input=output
    y=current_input
    cost=tf.reduce_sum(tf.square(y-x_tensor))
    with tf.name_scope('training'):
        cost=tf.reduce_sum(tf.square(y-x_tensor))
        optimizer=tf.train.AdamOptimizer(epsilon=0.001)
        train_op=optimizer.minimize(cost)
    auto_saver=tf.train.Saver()
    return {'x': x_tensor,'z':z,'y': y, 'cost': cost,'train':train_op,'saver':auto_saver}

    
    

In [ ]:
at=autoencoder() # build the computational graph for the convolutional autoencoders

In [15]:

#visualize the computational graph
from IPython.display import clear_output, Image, display, HTML
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def


def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
show_graph(tf.get_default_graph())

In [2]:
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images

In [3]:
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels

In [4]:
train_X=read_all_images('train_X.bin')
train_Y=read_labels('train_y.bin')
test_X=read_all_images('test_X.bin')
test_Y=read_labels('test_y.bin')
unlabeled=read_all_images('unlabeled_X.bin')

In [5]:
from random import sample

def prepare_batch_auto(X,batch_size):
    
    """
    Funtion to load images batch by batch
    
    """
    length=X.shape[0]
    Index=list(range(length))
    sampled_index=sample(Index,batch_size)
    sampled_images=X[sampled_index,:,:,:]
    x_batch=sampled_images
    return x_batch 



def prepare_batch_2(X,Y,batch_size):
    
    """
    Funtion to load images batch by batch
    
    """
    length=len(Y)
    Index=list(range(length))
    sampled_index=sample(Index,batch_size)
    sampled_images=X[sampled_index,:,:,:]
    x_batch=sampled_images
    y_batch=Y[sampled_index]
    return x_batch,y_batch 




In [ ]:
init=tf.global_variables_initializer()
n_epochs=10
batch_size=20
iterations_per_epoch=5000

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        print('Epoch',epoch, end='')
        for iters in range(iterations_per_epoch):
            x_batch=prepare_batch_auto(unlabeled,batch_size)
            sess.run(at['train'],feed_dict={at['x']:x_batch})
        print(' reconstruction_loss:',at['cost'].eval(feed_dict={at['x']:x_batch}))
    save_path=at['saver'].save(sess,'./Save/stl_auto.ckpt')
            
    

In [6]:
# Load computational graph for the trained conv_autoencoder

with tf.Session() as sess:
    Saver=tf.train.import_meta_graph('./Save/stl_auto.ckpt.meta')
    
    

In [7]:
# Adding fully a connected layer and final classifier
graph = tf.get_default_graph()
y=tf.placeholder(tf.int32,shape=[None])
w1 = graph.get_tensor_by_name("encode_2/Elu:0")

In [8]:
w1_flat=tf.reshape(w1,[-1,12*12*256])

In [9]:

with tf.name_scope('new_layers'):
    prelogits=tf.layers.dense(w1_flat,4096,activation=tf.nn.elu,name='prelogits')
    Logits=tf.layers.dense(prelogits,10,name='stl_logits')

In [10]:
with tf.name_scope('classifier'):
    entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=Logits,labels=y)
    loss=tf.reduce_mean(entropy)
    optimizer=tf.train.AdamOptimizer(epsilon=1)
    Vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='stl_logits')+tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='prelogits')
    train_op=optimizer.minimize(loss,var_list=Vars)

# Add evaluation operation node    
with tf.name_scope('eval'):
    correct=tf.nn.in_top_k(Logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))   
    


In [11]:
# Preprocess test sets
X_test,Y_test=prepare_batch_2(test_X,test_Y,8000)

In [14]:
# Traina and Evaluate the model
init=tf.global_variables_initializer()
stl_save=tf.train.Saver()
n_epochs=100
batch_size=50
n_iterations_per_epoch=100


with tf.Session() as sess:
    sess.run(init)
    Saver.restore(sess,'./Save/stl_auto.ckpt')
    
    for epoch in range(n_epochs):
        print('Epoch',epoch, end='')
        for iteration in range(n_iterations_per_epoch):
            print('.',end='')
            x_batch,y_batch=prepare_batch_2(train_X,train_Y,batch_size)
            y_eval=y_batch-1 
            sess.run(train_op, feed_dict={'x:0':x_batch,y:y_eval})
        acc_train=accuracy.eval(feed_dict={'x:0':x_batch,y:y_eval})
        print(' Train accuracy:', acc_train)
        
        save_path=stl_save.save(sess, './Save/STL_10')
    Y_eval=Y_test-1
    print('Computing final accuracy on the test set...')
    test_ac=np.zeros((1,100))
    for i in range(100):
        acc_test = accuracy.eval(feed_dict={'x:0': X_test[i*80:(i+1)*80,:,:,:], y: Y_eval[i*80:(i+1)*80]})
        test_ac[:,i]=acc_test
    print('Test accuracy:', np.mean(test_ac))
    

Epoch 0.................................................................................................... Train accuracy: 0.48
Epoch 1.................................................................................................... Train accuracy: 0.3
Epoch 2.................................................................................................... Train accuracy: 0.48
Epoch 3.................................................................................................... Train accuracy: 0.5
Epoch 4.................................................................................................... Train accuracy: 0.46
Epoch 5.................................................................................................... Train accuracy: 0.64
Epoch 6.................................................................................................... Train accuracy: 0.46
Epoch 7............................................................................................

Epoch 63.................................................................................................... Train accuracy: 0.92
Epoch 64.................................................................................................... Train accuracy: 0.88
Epoch 65.................................................................................................... Train accuracy: 0.92
Epoch 66.................................................................................................... Train accuracy: 1.0
Epoch 67.................................................................................................... Train accuracy: 0.94
Epoch 68.................................................................................................... Train accuracy: 0.98
Epoch 69.................................................................................................... Train accuracy: 0.96
Epoch 70...................................................................................